# Getting the Data into a SQL Database

This notebook takes CSV that contains meta features and information about complaints and loads it into a PostgreSQL database.

First, need to load the required packages.

In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
from sklearn.externals import joblib

In [10]:
# Set Postgres credentials
db_name = 'complaint1'
username = 'postgres'
host = 'localhost'
port = '5432' 
#password = ''

In [11]:
engine = create_engine( 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, db_name) )

In [12]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [13]:
# read a database from CSV and load it into a pandas dataframe
complaint = pd.read_csv('nars_clean_meta_28jan.csv', index_col=0)

In [14]:
complaint.to_sql(
    name=db_name, 
    con=engine,
    if_exists='append'
)

In [16]:
#connect to make queries using psycopg2
con = None
con = psycopg2.connect(database=db_name, 
    host='localhost',
    user=username,
    password=password)

# query:
sql_query = """
SELECT * FROM complaint1;
"""
test_data_from_sql = pd.read_sql_query(sql_query,con)
test_data_from_sql.head()

,level_0,index,date_rec,prod,subprod,issue,sub_issue,narrative,pub_resp,company,...,PRON,PROPN,PUNCT,SPACE,SYM,VERB,X,avg_words_sent,num_sent,num_word
0,0,177,01/11/2019,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,None,wells fargo charging exorbitant amount fees at...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,...,37.0,94.0,62,NaN,2.0,144.0,NaN,34.545455,22,760
1,1,186,01/11/2019,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,None,"filed several complaints wells fargo, opposing...",Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,...,7.0,27.0,26,NaN,1.0,38.0,NaN,30.125000,8,241
2,2,188,01/11/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,negative remark report recognize,Company has responded to the consumer and the ...,"HCFS Health Care Financial Services, Inc.",...,1.0,NaN,2,NaN,NaN,2.0,NaN,12.000000,1,12
3,3,189,01/11/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,"xx xx 2016, debt 160 00 appeared credit report...",Company believes it acted appropriately as aut...,Penn Credit Corporation,...,18.0,10.0,26,NaN,2.0,45.0,NaN,21.333333,12,256
4,4,195,01/11/2019,Mortgage,VA mortgage,Trouble during payment process,None,mortgage xxxx sold flagstar tried process firs...,Company has responded to the consumer and the ...,"FLAGSTAR BANK, FSB",...,22.0,5.0,19,1.0,NaN,48.0,NaN,22.444444,9,202


In [17]:
test_data_from_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356699 entries, 0 to 356698
Data columns (total 41 columns):
level_0            356699 non-null int64
index              356699 non-null int64
date_rec           356699 non-null object
prod               356699 non-null object
subprod            304629 non-null object
issue              356699 non-null object
sub_issue          246916 non-null object
narrative          356698 non-null object
pub_resp           173557 non-null object
company            356699 non-null object
state              355349 non-null object
zip                277815 non-null object
Tags               61578 non-null object
consent            356699 non-null object
how_submit         356699 non-null object
date_to_company    356699 non-null object
comp_resp          356699 non-null object
timely_resp        356699 non-null object
cons_disp          162981 non-null object
id                 356699 non-null int64
response           356699 non-null object
sentiment  

Looks like it was successful. Now we can use this database to fit and train models.